<a href="https://colab.research.google.com/github/GalSerge/methods_ml_2/blob/main/ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Упражнение 2
Напишите классификатор MNIST, который обучается до точности 99% или выше и делает это без фиксированного числа эпох - то есть вы должны прекратить обучение, как только достигнете этого уровня точности.

1. Этого надо достигнуть менее чем за 10 эпох, поэтому можно установить epoches=10, но не более.
2. Когда точность станет 99% или больше, надо распечатать строку «Достигнута точность 99%, поэтому обучение закончено!»


In [24]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from os import path, getcwd, chdir

mnist = tf.keras.datasets.mnist

In [33]:
class StopOnPoint(tf.keras.callbacks.Callback):
    def __init__(self, point):
        super(StopOnPoint, self).__init__()
        self.point = point

    def on_epoch_end(self, epoch, logs=None):
        accuracy = logs['accuracy']
        if accuracy >= self.point:
            self.model.stop_training = True

In [36]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape((60000, 28, 28, 1))
x_train = x_train.astype('float32') / 255
x_test = x_test.reshape((10000, 28, 28, 1))
x_test = x_test.astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', baseline=0.8)
model.fit(x_train, y_train, epochs=10, batch_size=100, callbacks=[StopOnPoint(0.99)])

print('Достигнута точность 99%, поэтому обучение закончено!')

test_loss, test_acc = model.evaluate(x_test, y_test)
print('Точность на тестовом наборе данных:', test_acc)

Epoch 1/10
600/600 [==============================] - 51s 83ms/step - loss: 0.2145 - accuracy: 0.9351
Epoch 2/10
600/600 [==============================] - 49s 81ms/step - loss: 0.0550 - accuracy: 0.9833
Epoch 3/10
600/600 [==============================] - 50s 83ms/step - loss: 0.0386 - accuracy: 0.9879
Epoch 4/10
600/600 [==============================] - 50s 83ms/step - loss: 0.0310 - accuracy: 0.9905
Достигнута точность 99%, поэтому обучение закончено!
313/313 [==============================] - 4s 13ms/step - loss: 0.0301 - accuracy: 0.9909
Точность на тестовом наборе данных: 0.9908999800682068
